In [1]:
import numpy as np

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # TODO: ques

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
from typing import List


class Function:
    def __call__(self, *input_variables: [Variable]) -> [Variable]:
        input_data = [input_variable.data for input_variable in input_variables]
        output_data = self.forward(*input_data)
        if not isinstance(output_data, tuple):
            output_data = (output_data,)
        output_variables = [Variable(as_array(data)) for data in output_data]
        for output_variable in output_variables:
            output_variable.set_creator(self)
        self.inputs: [Variable] = input_variables
        self.outputs: [Variable] = output_variables
        return output_variables if len(output_variables) > 1 else output_variables[0]
    
    def forward(self, input_data):
        raise NotImplementedError()
    
    def backward(self, output_data) -> [Variable]:
        """
        
        :param output_data: gradient of the output with respect to input data.
        :return: 
        """
        raise NotImplementedError()

In [5]:
class Add(Function):
    def forward(self, x0, x1):  # TODO: it seems not correct
        y = x0 + x1
        return (y,)

In [6]:
def add(x0, x1):
    return Add()(x0, x1)

In [7]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5
